<a href="https://colab.research.google.com/github/VineetMalik14/Causal-Structure-Learning-Survey/blob/main/discrete_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pgmpy numpy networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import networkx as nx
from pgmpy.models import BayesianModel
from pgmpy.sampling import BayesianModelSampling
from pgmpy.factors.discrete import TabularCPD
import random
import pandas as pd
import os

In [6]:
def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

def _graph_to_adjmat(G):
    return nx.to_numpy_array(G)

def _random_permutation(M):
    # np.random.permutation permutes first axis only
    P = np.random.permutation(np.eye(M.shape[0]))
    return P.T @ M @ P

def _random_acyclic_orientation(B_und):
    B = np.tril(_random_permutation(B_und), k=-1)
    B_perm = _random_permutation(B)
    return B_perm

def erdos_renyi(n_nodes, n_edges, seed=None):
    assert n_nodes > 0
    set_random_seed(seed)
    # Erdos-Renyi
    creation_prob = (2 * n_edges) / (n_nodes ** 2)
    G_und = nx.erdos_renyi_graph(n=n_nodes, p=creation_prob, seed=seed)
    B_und = _graph_to_adjmat(G_und)
    B = _random_acyclic_orientation(B_und)
    return B

def generate_random_dag(num_nodes, num_edges):
    G = nx.DiGraph()
    G.add_nodes_from(range(num_nodes))
    
    while G.number_of_edges() < num_edges:
        u, v = np.random.choice(num_nodes, size=2, replace=False)
        if u == v or G.has_edge(u, v):
            continue
        G.add_edge(u, v)
        if not nx.is_directed_acyclic_graph(G):
            G.remove_edge(u, v)
            
    return G

def generate_random_cpd(variable, variable_card, evidence, evidence_cards):
    if len(evidence) == 0:
        probabilities = np.random.dirichlet(np.ones(variable_card))
        probabilities = np.array([probabilities])
        probabilities = probabilities.T
        cpd = TabularCPD(variable, variable_card, probabilities)
    else:
        num_rows = variable_card
        num_cols = np.prod(evidence_cards)
        probabilities = np.random.dirichlet(np.ones(num_rows), size=num_cols)
        cpd = TabularCPD(variable, variable_card, probabilities.T,
                         evidence=evidence, evidence_card=evidence_cards)
    return cpd

In [13]:
def generate_discrete_data(num_nodes, num_edges, num_samples, seed):
    # Set the number of categories for each variable (assuming the same for all)
    variable_card = 2

    # Generate a random DAG
    # random_dag = generate_random_dag(num_nodes, num_edges)

    # seed = random.randint(0,9)
    print(num_nodes, num_edges, num_samples, seed)
    random_dag = erdos_renyi(n_nodes=num_nodes, n_edges=num_edges, seed=seed)
    random_dag = nx.from_numpy_array(random_dag, create_using=nx.DiGraph)
    print(type(random_dag))
    edges = list(random_dag.edges)

    # Create a Bayesian network with the random DAG
    model = BayesianModel(edges)
    model.add_nodes_from(random_dag.nodes)

    # Generate random CPD tables and add them to the Bayesian network
    for node in random_dag.nodes:
        parent_nodes = list(random_dag.predecessors(node))
        parent_cards = [variable_card] * len(parent_nodes)
        cpd = generate_random_cpd(node, variable_card, parent_nodes, parent_cards)
        model.add_cpds(cpd)

    # Verify if the model is consistent
    assert model.check_model()

    # Instantiate the sampling class and generate samples
    inference = BayesianModelSampling(model)
    synthetic_data = inference.forward_sample(size=num_samples)

    causal_data = pd.DataFrame(nx.to_numpy_array(random_dag))
    return synthetic_data, causal_data

In [15]:
FOLDER_PATH  = "./discrete_data_new/"

for n in [10, 30, 50]:
    for density in [1, 1.5]:
        p = pow(n, density-2)
        n_edges = int(n*n*p/2)
        max_sample_size = 4000
        for seed in range(3):
            X, true_causal_matrix = generate_discrete_data(n, n_edges, max_sample_size, seed)
            for sample_size in [1000, 4000]:
                file_name = "_d_{}_{}_{}_{}.csv".format(n, density, sample_size, seed) 
                true_causal_matrix.to_csv(os.path.join(FOLDER_PATH, "causal"+file_name), index=False)
                sample_sized_data = X
                sample_sized_data = sample_sized_data.head(sample_size)
                sample_sized_data.to_csv(os.path.join(FOLDER_PATH, "sim_data"+file_name), index=False)

10 5 4000 0
<class 'networkx.classes.digraph.DiGraph'>


/usr/local/lib/python3.10/dist-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

10 5 4000 1
<class 'networkx.classes.digraph.DiGraph'>


/usr/local/lib/python3.10/dist-packages/pgmpy/utils/mathext.py:82: UserWarning: Probability values don't exactly sum to 1. Differ by: 1.1102230246251565e-16. Adjusting values.
  warn(
/usr/local/lib/python3.10/dist-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/pgmpy/utils/mathext.py:82: UserWarning: Probability values don't exactly sum to 1. Differ by: 1.1102230246251565e-16. Adjusting values.
  warn(
/usr/local/lib/python3.10/dist-packages/pgmpy/utils/mathext.py:82: UserWarning: Probability values don't exactly sum to 1. Differ by: 2.220446049250313e-16. Adjusting values.
  warn(
/usr/local/lib/python3.10/dist-packages/pgmpy/utils/mathext.py:82: UserWarning: Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
  warn(


50 176 4000 2
<class 'networkx.classes.digraph.DiGraph'>


/usr/local/lib/python3.10/dist-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pgmpy/utils/mathext.py:82: UserWarning: Probability values don't exactly sum to 1. Differ by: 1.1102230246251565e-16. Adjusting values.
  warn(
/usr/local/lib/python3.10/dist-packages/pgmpy/utils/mathext.py:82: UserWarning: Probability values don't exactly sum to 1. Differ by: 2.220446049250313e-16. Adjusting values.
  warn(
/usr/local/lib/python3.10/dist-packages/pgmpy/utils/mathext.py:82: UserWarning: Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
  warn(
